<a href="https://colab.research.google.com/github/SChoi005/DeepLearningBasics/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

* **Large** and **different** dataset -> **Fine tuning**
* **Large** and **similar** dataset -> **Fine tuning**
* **Small** and **different** dataset -> **Fine tuning**
* **Small** and **similar** dataset -> **Transfer Learning**

## Downloading the Dogs vs Cats dataset

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2022-07-02 13:37:37--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.135.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   274MB/s    in 0.2s    

2022-07-02 13:37:37 (274 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Data Preprocessing

In [2]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.8.2'

### Unzipping the Dogs vs Cats dataset

In [3]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [4]:
zip_object = zipfile.ZipFile(dataset_path, "r")

In [5]:
zip_object.extractall("./")

In [6]:
zip_object.close()

### Setting up dataset paths

In [7]:
dataset_path_new =  "./cats_and_dogs_filtered/"

In [8]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre_trained model (MobileNetV2)

In [9]:
IMG_SHAPE = (128, 128, 3)

with tf.device('/device:GPU:0'):
  base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [10]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

### Freezing the baes model
* 초기가중치 예방

In [11]:
base_model.trainable = False

### Defining the custom head for our network

In [12]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [13]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [14]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [15]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [16]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

## Compiling the model
* 사전학습 네트워크를 사용하므로 기본값보다 작은 학습률을 지정해야함 옵티마이저를 문자열로 지정하면 모든기본인자를 가져감 이경우 학습률 파라미터를 정의해야함

In [18]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## Creating Data Generator

In [19]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [20]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [21]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


## Training the model

In [23]:
model.fit_generator(train_generator, epochs=20, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/20
16/16 [==============================] - 8s 531ms/step - loss: 0.3899 - accuracy: 0.8485 - val_loss: 0.3722 - val_accuracy: 0.8510
Epoch 2/20
16/16 [==============================] - 8s 519ms/step - loss: 0.3482 - accuracy: 0.8690 - val_loss: 0.3320 - val_accuracy: 0.8810
Epoch 3/20
16/16 [==============================] - 8s 517ms/step - loss: 0.3133 - accuracy: 0.8900 - val_loss: 0.2991 - val_accuracy: 0.8960
Epoch 4/20
16/16 [==============================] - 8s 507ms/step - loss: 0.2843 - accuracy: 0.9035 - val_loss: 0.2717 - val_accuracy: 0.9110
Epoch 5/20
16/16 [==============================] - 8s 511ms/step - loss: 0.2607 - accuracy: 0.9140 - val_loss: 0.2484 - val_accuracy: 0.9200
Epoch 6/20
16/16 [==============================] - 8s 501ms/step - loss: 0.2400 - accuracy: 0.9210 - val_loss: 0.2283 - val_accuracy: 0.9310
Epoch 7/20
16/16 [==============================] - 8s 506ms/step - loss: 0.2227 - accuracy: 0.9270 - val_loss: 0.2116 - val_accuracy: 0.9330
Epoch 

# Transfer learning model evaluation

In [24]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [25]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9549999833106995


# Fine tuning

## Un-freeze a few top layers from the model

In [26]:
base_model.trainable = True

In [27]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [28]:
fine_tune_at = 100

In [29]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

## Compiling the model for fine-tuning

In [30]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [31]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/5
16/16 [==============================] - 14s 581ms/step - loss: 0.1620 - accuracy: 0.9305 - val_loss: 0.0759 - val_accuracy: 0.9700
Epoch 2/5
16/16 [==============================] - 8s 525ms/step - loss: 0.0222 - accuracy: 0.9975 - val_loss: 0.0714 - val_accuracy: 0.9670
Epoch 3/5
16/16 [==============================] - 8s 514ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.0749 - val_accuracy: 0.9730
Epoch 4/5
16/16 [==============================] - 8s 527ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0913 - val_accuracy: 0.9730
Epoch 5/5
16/16 [==============================] - 8s 525ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.1116 - val_accuracy: 0.9700


## Evaluating the fine tuned model

In [32]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [34]:
print("Accuracy after fine tuning: {}".format(valid_accuracy))

Accuracy after fine tuning: 0.9700000286102295
